## What is Dimentionality Reduction
1. Reducing the number of features
2. Extracting the useful features that add more information
3. Feature selection also can be used to achieve dimentionality reduction

## Feature selection
The feature selection approach tries to find the subset of input variables. The three strategies are
1. The filter strategy ex Information gain
2. The wrapper strategy ex search guided by accuracy and
3. The Embedded strategy ex selected features added or removed while building the model based on prediction errors. 
